In [11]:
import psycopg2
import pandas as pd

In [12]:
def create_database():
    # connect with default database
    conn = psycopg2.connect(host='127.0.0.1', port ='5432', dbname = 'postgres',  user='postgres',  password='')
    conn.set_session(autocommit=True)
    cur = conn.cursor()
    # create pizza_db
    cur.execute("DROP DATABASE IF EXISTS pizza_db")
    cur.execute("CREATE DATABASE pizza_db")
    
    #closee connection with default database
    conn.close()
    
    #connect to pizza_db
    conn = psycopg2.connect(host='127.0.0.1', port ='5432', dbname = 'pizza_db',  user='****',  password='')
    cur = conn.cursor()
    
    return cur,conn

    

In [13]:
def drop_tables(cur, conn):
    for query in drop_table_quries:
        cur.execute(query)
        conn.commit()

### import csv data  

In [14]:
pizza_types_df = pd.read_csv("pizza_types.csv",encoding = 'unicode_escape')
order_details_df = pd.read_csv("order_details.csv",encoding = 'unicode_escape')
orders_df = pd.read_csv("orders.csv",encoding = 'unicode_escape')
pizza_df = pd.read_csv("pizzas.csv",encoding = 'unicode_escape')

In [15]:
pizza_types_df.head(10)

,pizza_type_id,name,category,ingredients
0,bbq_ckn,The Barbecue Chicken Pizza,Chicken,"Barbecued Chicken, Red Peppers, Green Peppers,..."
1,cali_ckn,The California Chicken Pizza,Chicken,"Chicken, Artichoke, Spinach, Garlic, Jalapeno ..."
2,ckn_alfredo,The Chicken Alfredo Pizza,Chicken,"Chicken, Red Onions, Red Peppers, Mushrooms, A..."
3,ckn_pesto,The Chicken Pesto Pizza,Chicken,"Chicken, Tomatoes, Red Peppers, Spinach, Garli..."
4,southw_ckn,The Southwest Chicken Pizza,Chicken,"Chicken, Tomatoes, Red Peppers, Red Onions, Ja..."
5,thai_ckn,The Thai Chicken Pizza,Chicken,"Chicken, Pineapple, Tomatoes, Red Peppers, Tha..."
6,big_meat,The Big Meat Pizza,Classic,"Bacon, Pepperoni, Italian Sausage, Chorizo Sau..."
7,classic_dlx,The Classic Deluxe Pizza,Classic,"Pepperoni, Mushrooms, Red Onions, Red Peppers,..."
8,hawaiian,The Hawaiian Pizza,Classic,"Sliced Ham, Pineapple, Mozzarella Cheese"
9,ital_cpcllo,The Italian Capocollo Pizza,Classic,"Capocollo, Red Peppers, Tomatoes, Goat Cheese,..."


In [16]:
order_details_df.head(10)

,order_details_id,order_id,pizza_id,quantity
0,1,1,hawaiian_m,1
1,2,2,classic_dlx_m,1
2,3,2,five_cheese_l,1
3,4,2,ital_supr_l,1
4,5,2,mexicana_m,1
5,6,2,thai_ckn_l,1
6,7,3,ital_supr_m,1
7,8,3,prsc_argla_l,1
8,9,4,ital_supr_m,1
9,10,5,ital_supr_m,1


In [17]:
orders_df.head(10)

,order_id,date,time
0,1,2015-01-01,11:38:36
1,2,2015-01-01,11:57:40
2,3,2015-01-01,12:12:28
3,4,2015-01-01,12:16:31
4,5,2015-01-01,12:21:30
5,6,2015-01-01,12:29:36
6,7,2015-01-01,12:50:37
7,8,2015-01-01,12:51:37
8,9,2015-01-01,12:52:01
9,10,2015-01-01,13:00:15


In [18]:
pizza_df.head(10)

,pizza_id,pizza_type_id,size,price
0,bbq_ckn_s,bbq_ckn,S,12.75
1,bbq_ckn_m,bbq_ckn,M,16.75
2,bbq_ckn_l,bbq_ckn,L,20.75
3,cali_ckn_s,cali_ckn,S,12.75
4,cali_ckn_m,cali_ckn,M,16.75
5,cali_ckn_l,cali_ckn,L,20.75
6,ckn_alfredo_s,ckn_alfredo,S,12.75
7,ckn_alfredo_m,ckn_alfredo,M,16.75
8,ckn_alfredo_l,ckn_alfredo,L,20.75
9,ckn_pesto_s,ckn_pesto,S,12.75


In [19]:
order_details_df = order_details_df.drop(['order_details_id'],axis =1)


In [20]:
print(order_details_df)

       order_id       pizza_id  quantity
0             1     hawaiian_m         1
1             2  classic_dlx_m         1
2             2  five_cheese_l         1
3             2    ital_supr_l         1
4             2     mexicana_m         1
...         ...            ...       ...
48615     21348  ckn_alfredo_m         1
48616     21348  four_cheese_l         1
48617     21348   napolitana_s         1
48618     21349     mexicana_l         1
48619     21350      bbq_ckn_s         1

[48620 rows x 3 columns]


### create the master dataframe joinning all dataframe together 

In [21]:
piz = order_details_df.merge(orders_df, on = ['order_id']).merge(pizza_df, on =['pizza_id'])\
    .merge(pizza_types_df, on =['pizza_type_id'])

In [22]:
piz.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 48620 entries, 0 to 48619
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   order_id       48620 non-null  int64  
 1   pizza_id       48620 non-null  object 
 2   quantity       48620 non-null  int64  
 3   date           48620 non-null  object 
 4   time           48620 non-null  object 
 5   pizza_type_id  48620 non-null  object 
 6   size           48620 non-null  object 
 7   price          48620 non-null  float64
 8   name           48620 non-null  object 
 9   category       48620 non-null  object 
 10  ingredients    48620 non-null  object 
dtypes: float64(1), int64(2), object(8)
memory usage: 4.5+ MB


In [39]:
piz

,order_id,pizza_id,quantity,date,time,pizza_type_id,size,price,name,category,ingredients
0,1,hawaiian_m,1,2015-01-01,11:38:36,hawaiian,M,13.25,The Hawaiian Pizza,Classic,"Sliced Ham, Pineapple, Mozzarella Cheese"
1,77,hawaiian_m,1,2015-01-02,12:22:46,hawaiian,M,13.25,The Hawaiian Pizza,Classic,"Sliced Ham, Pineapple, Mozzarella Cheese"
2,146,hawaiian_m,1,2015-01-03,14:22:10,hawaiian,M,13.25,The Hawaiian Pizza,Classic,"Sliced Ham, Pineapple, Mozzarella Cheese"
3,163,hawaiian_m,1,2015-01-03,16:54:54,hawaiian,M,13.25,The Hawaiian Pizza,Classic,"Sliced Ham, Pineapple, Mozzarella Cheese"
4,247,hawaiian_m,1,2015-01-04,20:55:29,hawaiian,M,13.25,The Hawaiian Pizza,Classic,"Sliced Ham, Pineapple, Mozzarella Cheese"
...,...,...,...,...,...,...,...,...,...,...,...
48615,21167,brie_carre_s,1,2015-12-27,17:39:57,brie_carre,S,23.65,The Brie Carre Pizza,Supreme,"Brie Carre Cheese, Prosciutto, Caramelized Oni..."
48616,21198,brie_carre_s,1,2015-12-28,15:39:02,brie_carre,S,23.65,The Brie Carre Pizza,Supreme,"Brie Carre Cheese, Prosciutto, Caramelized Oni..."
48617,21225,brie_carre_s,1,2015-12-29,13:20:17,brie_carre,S,23.65,The Brie Carre Pizza,Supreme,"Brie Carre Cheese, Prosciutto, Caramelized Oni..."
48618,21282,brie_carre_s,1,2015-12-31,12:09:59,brie_carre,S,23.65,The Brie Carre Pizza,Supreme,"Brie Carre Cheese, Prosciutto, Caramelized Oni..."


### check null value in dataframe

In [23]:
piz.isna().sum()

order_id         0
pizza_id         0
quantity         0
date             0
time             0
pizza_type_id    0
size             0
price            0
name             0
category         0
ingredients      0
dtype: int64

### Create data model for the dataframe

#### create table and insert data

In [24]:
cur, conn = create_database()

In [25]:
order_table_create = ("CREATE TABLE IF NOT EXISTS order_t(order_id INT, date VARCHAR, time VARCHAR)")
cur.execute(order_table_create)
conn.commit()

In [26]:
pizza_table_create = ("CREATE TABLE IF NOT EXISTS pizza_t(pizza_id VARCHAR, pizza_type_id VARCHAR, size VARCHAR, price FLOAT)")
cur.execute(pizza_table_create)
conn.commit()


In [27]:
order_details_table_create = ("CREATE TABLE IF NOT EXISTS order_details(order_id INT, pizza_id VARCHAR, quantity INT)")
cur.execute(order_details_table_create)
conn.commit()

In [28]:
pizza_types_table_create = ("CREATE TABLE IF NOT EXISTS pizza_types(pizza_type_id VARCHAR, name VARCHAR, category VARCHAR, ingredients VARCHAR)")
cur.execute(pizza_types_table_create)
conn.commit()

In [29]:
### insert data to the tabelb

In [30]:
order_table_insert = (" INSERT INTO order_t(order_id, date, time) VALUES(%s, %s, %s)")

In [31]:
for i, row in orders_df.iterrows():
    cur.execute(order_table_insert, list(row))
    conn.commit()

In [32]:
pizza_table_insert = (" INSERT INTO pizza_t(pizza_id, pizza_type_id, size, price) VALUES(%s, %s, %s, %s)")

In [33]:
for i, row in pizza_df.iterrows():
    cur.execute(pizza_table_insert, list(row))
    conn.commit()


In [34]:
order_details_table_insert = (" INSERT INTO order_details(order_id, pizza_id, quantity) VALUES(%s, %s, %s)")

In [35]:
for i, row in order_details_df.iterrows():
    cur.execute(order_details_table_insert, list(row))
    conn.commit()

In [36]:
pizza_types_table_insert = ("INSERT INTO pizza_types(pizza_type_id, name, category, ingredients) VALUES(%s, %s, %s, %s) ")

In [37]:
for i, row in pizza_types_df.iterrows():
    cur.execute(pizza_types_table_insert, list(row))
    conn.commit()

In [38]:
cur.close()
conn.close()
print("close your cursor and connection.")

close your cursor and connection.
